In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
import numpy as np
import random
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, Embedding, Lambda,Activation,Dropout
from sklearn.utils import shuffle
import numpy as np
from keras import layers
from keras.models import Model
import scipy
from scipy import sparse
from sklearn.decomposition import TruncatedSVD
import pickle
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import matplotlib.cm as cm

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = 'drive/My Drive/reviews_Electronics_5.json.gz'

In [ ]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF(file_path)

In [ ]:
data = df['reviewText']
data = data.values
len(data)

1689188

In [ ]:
def clean_data_1(data,text,tokens,i):
	punc = ('(', ')', '?', ':', ';', ',', '.', '!', '/', '-','"', "'", '{', '}', '[', ']', '_', '+', '=')
	newtext = ""
	for w in text.split(' '):
		if len(w)>=2 and w[-1] in punc and type(w)!=list:
			w = w[:-1]+" "+w[-1]
		newtext+=w+" "
            
	tweet = newtext.strip()
                        
	data[i] = text
	tokens.extend(data[i].split())

def clean_data(data,tokens):	

	for i in range(len(data)):
		encoded_string = data[i].encode("ascii", "ignore")

		text = encoded_string.decode()
		text = text.strip()
		text = re.sub(r'\n',' ',text)
		specialchars = ('{','}','!','#','$','%','^','&','*','(',')','<','>','+','-','/','~','`',':',';','_','=','|','?','@','[',']')
            
		text = re.sub(r'http?://[\n\S]+', '<URL>',text)
		text = re.sub(r'#[\n\S]+\b', '<HASHTAG>',text)
		text = re.sub(r'@[\n\S]+\b', '<MENTION>',text)

		for k in specialchars:  ## remove duplicate characters 
			text = re.sub(r'\{}+'.format(k),k,text)

		clean_data_1(data,text,tokens,i)


def Tokenize(data):
	data = re.split('(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s',data)
	tokens = []
	clean_data(data,tokens)
	return tokens

In [ ]:
def tokenize(d):
  d = re.sub(r'[,!?;-]+', '.', d)
  d = nltk.word_tokenize(d)  # tokenize string to words
  d = [ ch.lower() for ch in d
  if ch.isalpha() #Check if all the characters in the text are letters
    or ch == '.']
  return d

def helper_fun(data):
  word2Ind , Ind2word , wordfreq = dict() ,dict() , dict()
  text_corpus = []

  a = 0
  for d in data:
    if a >=20000:
      break
    a+=1
    d = tokenize(d)
    #d = Tokenize(d)
    text_corpus.extend(d)
  
  vocab = sorted(list(set(text_corpus)))

  for word in text_corpus:
    if word in wordfreq:
      wordfreq[word]+=1
    else:
      wordfreq[word]=1

  idx = 0    
  for word in vocab:
    word2Ind[word]=idx
    Ind2word[idx]=word
    idx+=1
  return word2Ind,Ind2word,wordfreq,text_corpus,vocab


word2Ind,Ind2word,wordfreq,text_corpus,vocab = helper_fun(data)
print(len(word2Ind),len(Ind2word),len(wordfreq),len(text_corpus),len(vocab))

29230 29230 29230 2202536 29230


In [ ]:
def get_windows(words, C):
    i = C
    while i < len(words) - C:
        center_word = words[i]
        context_words = words[(i - C):i] + words[(i+1):(i+C+1)]
        yield context_words, center_word
        i += 1

        
def generate_context_word(text_corpus,C):
  context_and_center_word = []
  for x,y in get_windows(text_corpus,2):
    x +=[y]
    context_and_center_word.append(x)
  return context_and_center_word
context_and_center_word = generate_context_word(text_corpus,2)

#1. Frequency based embedding:
##Co-Occurrence Matrix with a fixed context window

#Co-occurance matrix

In [ ]:
def find_co_occurence_matrix(text_corpus,word2Ind,context_and_center_word):
    
    co_occurrence_matrix = None
    co_occurrence_matrix = sparse.lil_matrix((len(word2Ind), len(word2Ind)),dtype=np.float64)
    print(co_occurrence_matrix.shape)

    for context in context_and_center_word:
      l1,l2,r1,r2,c = context
      co_occurrence_matrix[word2Ind[l1],word2Ind[c]] +=1.0
      co_occurrence_matrix[word2Ind[l2],word2Ind[c]] +=1.0
      co_occurrence_matrix[word2Ind[c],word2Ind[r1]] +=1.0
      co_occurrence_matrix[word2Ind[c],word2Ind[r1]] +=1.0 
    co_occurrence_matrix.setdiag(1.0)
    return co_occurrence_matrix

In [ ]:
co_occurence_matrix = find_co_occurence_matrix(text_corpus,word2Ind,context_and_center_word)
co_occurence_matrix


(29230, 29230)


<29230x29230 sparse matrix of type '<class 'numpy.float64'>'
	with 869435 stored elements in List of Lists format>

In [ ]:
with open('co_occurrence', 'wb') as fp:
    pickle.dump(co_occurence_matrix, fp)

#SVD on co occurence matrix


In [ ]:
def svd(co_occurence_matrix):
    sum_variance = []
    params = range(10,100,10)
    for p in params:
        svd = TruncatedSVD(n_components=p)
        svd.fit(co_occurence_matrix)
        var = round(svd.explained_variance_ratio_.sum(),3)
        sum_variance.append(var)

    index_max = sum_variance.index(max(sum_variance))
    bestdim = params[index_max]
    svd = TruncatedSVD(n_components=p)
    X = svd.fit(co_occurence_matrix)
    return X,bestdim


X_trunc,bestd  = svd(co_occurence_matrix)


In [ ]:
svd = TruncatedSVD(n_components=bestd)
X_trunc = svd.fit_transform(co_occurence_matrix)

In [ ]:
with open('svd', 'wb') as fp:
    pickle.dump(X_trunc, fp)

#Analysis

In [ ]:
sample_words = ['always','basement', 'darkest','leader','pay']

In [ ]:
def compute_final_words(vec,id,num,Ind2word):
  sim = []
  words = []
  for idx in range(0,len(vocab)):
    if idx!=id:
      next = X_trunc[idx]
      sim.append(1.0 - scipy.spatial.distance.cosine(next,vec))
      words.append(Ind2word[idx])
  nearest = sorted(zip(sim, words),reverse=True)[:num]
  final_words = []
  for sim,w in nearest:
    final_words.append(w)
    print(w,sim)
  print()
  return final_words

def nearest_words(vocab,word2Ind,Ind2word,word,num=10):
  if word not in vocab:
    print("given word not present in the input ")
  else:
    id  = word2Ind[word]
    vec = X_trunc[id]
    return compute_final_words(vec,id,num,Ind2word)


In [ ]:
similar = dict()
similar_word = []
for w in sample_words:
  similar[w] = nearest_words(vocab,word2Ind,Ind2word,w,num=10)
  similar_word.extend(similar[w])

already 0.8410825033120323
still 0.8180427427526878
not 0.7871899692274181
actually 0.7801587666685462
also 0.7356777479116572
have 0.7292689413118997
never 0.7258792150491332
typically 0.7205328379872039
usually 0.7168764260739218
has 0.709146114244936

laptop 0.9902291375891341
games 0.9839854575305471
computer 0.9819193010938787
hdtv 0.979148134693322
memos 0.9786831520088677
modem 0.9783863630526972
garage 0.978365070386736
laptops 0.9775779014183799
phone 0.9766952189421442
closet 0.9765470816725937

nailing 0.9663156840695792
blaming 0.9478921448347581
grandsons 0.9457968676328915
raises 0.9123343124673489
inaugurated 0.8781806993601368
harmless 0.8513908945604156
glancing 0.8506660715734019
deepanker 0.8506601562125014
alarmed 0.8506557211076967
worship 0.8506534249672413

jagged 0.984783168669845
pride 0.983229721513904
bult 0.9828754803496395
sneaks 0.9828706157953515
iterested 0.9828693184536867
inching 0.9828691638760354
soaked 0.9822190414168616
explicit 0.9804411103625297


10 nearest word for word 'camera' using word embedding model

In [ ]:
similar[w] = nearest_words(vocab,word2Ind,Ind2word,'camera',num=10)
similar_word.extend(similar[w])

antenna 0.994439742472162
lens 0.992465781238871
palm 0.9920087607973263
receiver 0.9919921981410513
player 0.9919531330879344
cradle 0.9917315379052744
reader 0.991482346196552
system 0.9913066438809595
hardware 0.9910211613073194
card 0.9906212489478338



#2. Prediction based embedding
## Continuous Bag of Words(CBOW) with negative sampling

#CBOW model

In [ ]:
totalWords = sum([freq**(3/4) for freq in wordfreq.values()])
print(totalWords)
wordProb = {word:(freq**(3/4))/totalWords for word, freq in wordfreq.items()}
word_pos_indices = {w:set() for word,w in word2Ind.items()}

ind2prob = {}
for word,id in word2Ind.items():
  ind2prob[id] = wordProb[word]

348604.00429825176


In [ ]:
def find_pos_sample(context_and_center_word,wordProb,word2Ind,word_pos_indices):
  context_c_vector = []
  for x in context_and_center_word:
    new_list=[]
    for y in x:
      new_list.append(wordProb[y])
      index_word = word2Ind[x[-1]]
      word_pos_indices[index_word].add(word2Ind[y])
    context_c_vector.append(new_list)
  return np.array(context_c_vector)

In [ ]:
positive_sample = find_pos_sample(context_and_center_word,wordProb,word2Ind,word_pos_indices)
print(len(positive_sample))

2202532


### preparing negative sample data 

In [ ]:
negative_samples = []
negative_keys = set()

def negIdx(word2Ind,word_pos_indices):
  n = len(word2Ind)
  pos_indices = sorted(list(word_pos_indices[id]))
  raw_samp = np.arange(0, n - len(pos_indices))
  pos_inds_adj = pos_indices - np.arange(len(pos_indices))
  ss = np.searchsorted(pos_inds_adj, raw_samp, side='right')
  neg_inds = raw_samp + ss
  return neg_inds

def fun(C,neg_inds,ind2prob):
  for j in range(k):
    sample = set()
    check_sample = []
    cnt = 0
    while(cnt<2*C):
      r = random.choice(neg_inds)
      if ind2prob[r] in sample:
        continue
      else:
        check_sample.append(r)
        sample.add(ind2prob[r])
        cnt+=1
    sample = list(sample)
    check_sample.sort()
    check_sample.append(id)
    sample.append(ind2prob[id])
    if tuple(check_sample) in negative_keys:
      continue
    else:
      negative_keys.add(tuple(check_sample))
      negative_samples.append(np.array(sample))


C = 2
count=0
k=50
for word,id in word2Ind.items():
  neg_inds = negIdx(word2Ind,word_pos_indices)
  fun(C,neg_inds,ind2prob)
  
  if count%10000==0 and count!=0:
    print("number of itereation :",count)
  count+=1
negative_samples = np.array(negative_samples)
len(negative_samples)

number of itereation : 10000
number of itereation : 20000


1461500

In [ ]:
X = np.concatenate([np.array(positive_sample), np.array(negative_samples)], axis=0)
y = np.concatenate([[1]*len(positive_sample), [0]*len(negative_samples)])

X, y = shuffle(X, y)
print(X.shape,y.shape)

(3664032, 5) (3664032,)


In [ ]:
def cbow_model(word2Ind,X,y):
  dim = [50,100,200]
  optim = ['adam','SGD','RMSprop']
  for a in dim:
    for op in optim:
      embedding_layer = layers.Embedding(len(word2Ind),a, 
                                    embeddings_initializer="RandomNormal",
                                    input_shape=(5,))
    
      cbow = Sequential([embedding_layer,layers.GlobalAveragePooling1D(),layers.Dense(1, activation='sigmoid'),])

      cbow.compile(optimizer= op,loss='binary_crossentropy',metrics=['accuracy'])
      cbow.fit(X,y, epochs=1,batch_size=10000, verbose=1)
cbow_model(word2Ind,X,y)

367/367 [==============================] - 27s 72ms/step - loss: 0.6728 - accuracy: 0.6006


In [ ]:
embedding_layer = layers.Embedding(len(word2Ind),50, 
                                    embeddings_initializer="RandomNormal",
                                    input_shape=(5,))
cbow = Sequential([embedding_layer,layers.GlobalAveragePooling1D(),layers.Dense(1, activation='sigmoid'),])
cbow.compile(optimizer= 'adam',loss='binary_crossentropy',metrics=['accuracy'])
cbow.summary()
cbow.fit(X,y, epochs=2,batch_size=10000, verbose=1)

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     (None, 5, 50)             1461500   
                                                                 
 global_average_pooling1d_9   (None, 50)               0         
 (GlobalAveragePooling1D)                                        
                                                                 
 dense_9 (Dense)             (None, 1)                 51        
                                                                 
Total params: 1,461,551
Trainable params: 1,461,551
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
367/367 [==============================] - 13s 34ms/step - loss: 0.6735 - accuracy: 0.6000
Epoch 2/2
367/367 [==============================] - 12s 34ms/step - loss: 0.6726 - accuracy: 0.6011


In [ ]:
weights = cbow.get_weights()[0]
print(weights)

[[ 4.03603651e-02  9.67170000e-02 -6.76979274e-02 ... -3.29332352e-02
  -7.66970441e-02  2.93372553e-02]
 [-2.96848884e-05  1.08904980e-01  4.08983231e-03 ... -5.47070168e-02
   9.27652568e-02 -4.27199863e-02]
 [-6.23065494e-02 -1.58598591e-02  3.58302109e-02 ... -4.10533249e-02
  -2.24872809e-02  1.94843449e-02]
 ...
 [-5.93487313e-03 -3.27911638e-02  3.02086901e-02 ... -3.78171913e-02
  -8.04222003e-02 -1.37602603e-02]
 [ 7.10816542e-03  2.47261990e-02 -1.59933548e-02 ...  6.10520504e-02
  -1.17819429e-04  1.19115645e-02]
 [ 3.65896076e-02 -2.11912226e-02 -7.49712512e-02 ...  3.67597640e-02
   9.49614309e-03 -4.52837944e-02]]


In [ ]:
embedding_layer(word2Ind['i'])

<tf.Tensor: shape=(50,), dtype=float32, numpy=
array([ 5.7349321e-02,  7.0858911e-02,  3.5229500e-02,  1.9962318e-02,
       -3.7097090e-03, -8.6336408e-04,  9.1481162e-03,  1.0296783e-02,
       -7.0964755e-03, -7.7855992e-03,  3.0016402e-02, -1.3823777e-02,
       -4.2098220e-02,  2.4534421e-02,  2.7217371e-03, -1.4737899e-02,
        6.9577865e-02,  2.9340279e-03,  5.3908911e-02,  5.0672092e-03,
        4.4184592e-02,  8.9348648e-03,  6.6944622e-02,  5.8305591e-02,
        1.7145732e-02, -1.9668451e-02, -2.9166970e-02, -2.3568740e-02,
       -1.0929397e-02,  3.0372519e-02,  3.4052726e-02,  4.6268377e-02,
        2.3129078e-02,  2.5998900e-02, -6.4781114e-02, -9.1607444e-02,
        4.7723337e-05,  2.6123330e-02, -6.3877837e-03, -5.2000802e-02,
       -1.3012047e-02,  1.0983963e-01,  5.6695309e-02,  1.2659338e-02,
       -2.1063795e-02,  5.3352721e-02, -1.0541013e-02,  4.7565352e-02,
        2.1364285e-02,  2.3299668e-02], dtype=float32)>

In [ ]:
with open('embeddings_cbow', 'wb') as fp:
    pickle.dump(embedding_layer, fp)

In [ ]:
cbow.save("cbow_model.h5")


In [ ]:
sample_words = ['always','basement', 'darkest','leader','pay']

In [ ]:
def compute_final_word_cbow(sim,vocab,num):
  similar = sorted(zip(sim, vocab),reverse=True)[:num]
  final_words = []
  for similarity, word in similar:
    final_words.append(word)
    print(word,similarity)
  print('')
  return final_words

def similarity_cbow(word2Ind,w,model,vocab,num=10):
  sim = []
  words = []
  vec = model(word2Ind[w])

  for w in vocab:
    wvec = model(word2Ind[w])
    sim.append(1.0 - scipy.spatial.distance.cosine(wvec,vec))
  return compute_final_word_cbow(sim,vocab,num)

In [ ]:
similarity = dict()
similar_word = []
for w in sample_words:
  similarity[w] =  similarity_cbow(word2Ind,w,embedding_layer,vocab,num=10)
  similar_word.extend(similarity[w])

always 1.0
outfits 0.5313782691955566
demagnetization 0.5117654800415039
ergonimaclly 0.5054018497467041
jukeboxes 0.5052174925804138
emagazines 0.4931686520576477
croaked 0.490444153547287
lithiums 0.4890112280845642
extracts 0.4781128168106079
quickoffice 0.4644654095172882

basement 1.0
inadequate 0.5313094854354858
ui 0.5118705630302429
human 0.49932152032852173
diskettes 0.4898269474506378
incl 0.482428640127182
touchnooter 0.48107320070266724
wheresmyrebate 0.4693569242954254
cx 0.4622194766998291
unreserved 0.4534768760204315

darkest 1.0
bleeding 0.5391285419464111
cook 0.5149738788604736
carter 0.5050573945045471
woofer 0.4817601442337036
sympathetic 0.4797098636627197
swirl 0.4783117473125458
mouse 0.46413734555244446
doublets 0.4619845747947693
guilty 0.45918598771095276

leader 1.0
gettingusb 0.5978992581367493
multilayer 0.5507175922393799
revel 0.5202323794364929
imaging 0.4736235439777374
dominican 0.4734865427017212
riva 0.47305166721343994
mkiii 0.4629632234573364
surg

10 nearest word for word 'camera' using cbow model

In [ ]:
similarity[w] =  similarity_cbow(word2Ind,'camera',embedding_layer,vocab,num=10)
similar_word.extend(similarity[w])

camera 1.0
geography 0.49916425347328186
award 0.49798595905303955
automatcially 0.49518898129463196
carryable 0.49262094497680664
settled 0.47833219170570374
higly 0.47606948018074036
actives 0.46999391913414
roundish 0.46856632828712463
adquate 0.464630663394928

